# Kaggle Challenge Model Training 

<br>

## **Baseline Model**
- ## [Load the training data](#loading-data)
- ## [Building a BoW(bag-of-word) classifer](#building-the-bow-model)
- ## [n-grams](#building-n-grams-model)
- ## [Building tf-idf document vectors](#building-tf-idf-model)
- ## [Word embeddings](#building-word-embeddings-model)

## **ML Model**
- ## [GS tf-idf LogisticRegression](#gs-tf-idf)
- ## [GS word embeddings LogisticRegression](#gs-word-embedding)
- ## [Clustering Model](#kmean-clustering)

## **Loading Data**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import string
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import warnings

warnings.filterwarnings('ignore')

RSEED = 42

In [2]:
df = pd.read_pickle("../data/preprocess_train.pkl")

In [3]:
df.head()

,id,keyword,location,text,target,text_clean,text_token,text_final,pos_tags,wordnet_tag,lemmatize,text_lemma,char_count1,char_count2,word_count1,word_count2,mword_leng1,mword_leng2
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this earthquake Ma...,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...","[(deeds, NNS), (reason, NN), (earthquake, NN),...","[(deeds, n), (reason, n), (earthquake, n), (ma...","[deed, reason, earthquake, may, allah, forgive...",deed reason earthquake may allah forgive u,68,42,13,7,4.307692,5.142857
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask Canada,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]","[(forest, JJS), (fire, NN), (near, IN), (la, J...","[(forest, a), (fire, n), (near, n), (la, a), (...","[forest, fire, near, la, ronge, sask, canada]",forest fire near la ronge sask canada,37,37,7,7,4.428571,4.428571
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked to shelter in place are be...,"[all, residents, asked, to, shelter, in, place...","[residents, asked, shelter, place, notified, o...","[(residents, NNS), (asked, VBD), (shelter, JJ)...","[(residents, n), (asked, v), (shelter, a), (pl...","[resident, ask, shelter, place, notify, office...",resident ask shelter place notify officer evac...,130,79,22,11,4.954545,6.272727
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...,"[13000, people, receive, wildfires, evacuation...","[13000, people, receive, wildfires, evacuation...","[(13000, CD), (people, NNS), (receive, JJ), (w...","[(13000, n), (people, n), (receive, a), (wildf...","[13000, people, receive, wildfire, evacuation,...",13000 people receive wildfire evacuation order...,63,57,8,7,6.875000,7.285714
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby Alaska as s...,"[just, got, sent, this, photo, from, ruby, ala...","[got, sent, photo, ruby, alaska, smoke, wildfi...","[(got, VBD), (sent, JJ), (photo, NN), (ruby, N...","[(got, v), (sent, a), (photo, n), (ruby, n), (...","[get, sent, photo, ruby, alaska, smoke, wildfi...",get sent photo ruby alaska smoke wildfires pou...,86,54,16,9,4.375000,5.111111


## **Building the BoW Model**

In [4]:
vectorizer = CountVectorizer()

X_train, X_test, y_train, y_test = train_test_split(df['text_lemma'], df['target'], test_size=0.2, random_state=RSEED)
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

clf = LogisticRegression(random_state=RSEED)

clf.fit(X_train_bow, y_train)
accuracy = clf.score(X_test_bow, y_test)
print(accuracy)

0.7872619829284307


## **Building n-grams Model**

In [5]:
vectorizer = CountVectorizer(ngram_range=(1,3))

X_train, X_test, y_train, y_test = train_test_split(df['text_lemma'], df['target'], test_size=0.2, random_state=RSEED)
X_train_ng = vectorizer.fit_transform(X_train)
X_test_ng = vectorizer.transform(X_test)

clf_ng = LogisticRegression(random_state=RSEED)

clf_ng.fit(X_train_ng, y_train)
accuracy = clf_ng.score(X_test_ng, y_test)
print(accuracy)

0.7918581746552856


## **Building tf-idf Model**

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=5)

X_train, X_test, y_train, y_test = train_test_split(df['text_lemma'], df['target'], test_size=0.2, random_state=RSEED)
X_train_tf = vectorizer.fit_transform(X_train)
X_test_tf = vectorizer.transform(X_test)

clf_tf = LogisticRegression(random_state=RSEED)

clf_tf.fit(X_train_tf, y_train)
accuracy = clf_tf.score(X_test_tf, y_test)
print(accuracy)

0.793827971109652


## **Building Word embeddings Model**

In [7]:
import spacy

In [8]:
nlp = spacy.load('en_core_web_lg')

def sent_vectorizer(sent):
    doc = nlp(sent)
    sent_vec =[]
    numw = 0
    for token in doc:
        try:
            if numw == 0:
                sent_vec = token.vector
            else:
                sent_vec = np.add(sent_vec, token.vector)
            numw+=1
        except:
            pass
    
    return np.asarray(sent_vec)
  
# Saving the embeddings in a list X
X=[]
for sentence in df['text_lemma']:
    X.append(sent_vectorizer(sentence))


In [9]:
print('Number of targets: ', len(df.target))
print('Number of X: ', len(X))

Number of targets:  7613
Number of X:  7613


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, df['target'], test_size=0.2, random_state=RSEED)

clf_ed = LogisticRegression(random_state=RSEED)

clf_ed.fit(X_train, y_train)
accuracy = clf_ed.score(X_test, y_test)
print(accuracy)

0.7931713722915299


### **_PCA for Dimension Reduction_**

In [11]:
# Using PCA for Dimensionality Reduction
# And the StandardScaler to scale the data 

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca_ = PCA(0.99, random_state=RSEED)
X_pca=pca_.fit_transform(X_scaled)

In [12]:
df_spacy = pd.read_pickle("../data/preprocess_train_spacy.pkl")

In [13]:
features = pd.DataFrame(X_pca)
#df_new = pd.concat([features, df['char_count2'], df_spacy['noun']], axis=1)
df_new = pd.concat([features, df['char_count2']], axis=1)

In [14]:
df_new.head()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,char_count2
0,1.393424,-5.001789,-1.911325,6.309400,2.383463,1.321009,1.548961,-0.290057,-0.355467,-0.166760,...,0.389286,0.323324,0.252453,-0.044470,0.011376,0.121080,0.080351,-0.125371,0.039218,42
1,-7.028380,-2.145287,-0.083162,-1.712946,3.596682,-1.248980,5.226906,3.112906,-2.214747,0.656909,...,-0.173009,-0.255258,0.425066,-0.436384,0.111585,0.337364,0.090446,-0.181701,-0.273838,37
2,2.774730,8.748291,-4.956767,3.196636,0.886435,2.278028,-2.225675,8.305098,1.743187,-4.573780,...,-0.299397,-0.307480,0.244903,-0.282708,-0.017390,0.240774,0.007214,-0.449125,0.111079,79
3,-3.431551,2.914703,-1.314137,3.855124,0.455495,0.908601,0.384746,2.608722,-0.489982,-1.816680,...,-0.114967,-0.032282,-0.246767,0.061446,-0.141505,0.019970,-0.063878,-0.118458,-0.166779,57
4,-0.032151,-1.687316,-0.492876,-4.201322,-1.692617,-0.729380,-2.052096,4.313179,0.020465,-0.379250,...,-0.167739,0.567429,0.148605,0.438867,-0.333850,-0.002970,0.344776,-0.562435,-0.291844,54


In [15]:
X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca, df['target'], test_size=0.2, random_state=RSEED)

clf_ed_pca = LogisticRegression(random_state=RSEED)

clf_ed_pca.fit(X_train_pca, y_train)
accuracy = clf_ed_pca.score(X_test_pca, y_test)
print(accuracy)

0.8063033486539725


In [16]:
X_train_pca2, X_test_pca2, y_train2, y_test2 = train_test_split(df_new, df['target'], test_size=0.2, random_state=RSEED)

clf_ed_pca2 = LogisticRegression(random_state=RSEED, max_iter=500)

clf_ed_pca2.fit(X_train_pca2, y_train2)
accuracy = clf_ed_pca2.score(X_test_pca2, y_test2)
print(accuracy)

0.8108995403808273


## **Grid Search for LG model**

### **_GS tf-idf_**

In [17]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, rand_score

In [18]:
lg_classifier = LogisticRegression(random_state=RSEED)

In [19]:
param_grid = {"penalty" : ["l2", "l1", "none", "elasticnet"],
              "solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"],
              "max_iter": [50, 100, 200, 300, 400, 500, 600, 1000]
             }

In [20]:
# we will do this at least twice.. according to DRY we should write a function
def print_pretty_summary(name, model, y_test, y_pred_test):
    print(name)
    print('=======================')
    print('penalty: {}'.format(model.penalty))
    print('solver: {}'.format(model.solver))
    print('max_iter: {}'.format(model.max_iter))
    accuracy = accuracy_score(y_test, y_pred_test)
    print('Test accuracy: {:2f}'.format(accuracy))
    return accuracy

In [21]:
gs_tf = GridSearchCV(lg_classifier, param_grid, scoring='accuracy',
                  cv=5, verbose=5, n_jobs=-1)

In [22]:
gs_tf.fit(X_train_tf, y_train)

Fitting 5 folds for each of 160 candidates, totalling 800 fits
[CV 2/5] END max_iter=50, penalty=l2, solver=lbfgs;, score=0.801 total time=   0.0s
[CV 3/5] END max_iter=50, penalty=l2, solver=lbfgs;, score=0.807 total time=   0.0s
[CV 2/5] END max_iter=50, penalty=l2, solver=newton-cg;, score=0.801 total time=   0.0s
[CV 3/5] END max_iter=50, penalty=l2, solver=newton-cg;, score=0.809 total time=   0.0s
[CV 1/5] END max_iter=50, penalty=l2, solver=lbfgs;, score=0.796 total time=   0.0s
[CV 4/5] END max_iter=50, penalty=l2, solver=newton-cg;, score=0.781 total time=   0.0s
[CV 4/5] END max_iter=50, penalty=l2, solver=lbfgs;, score=0.781 total time=   0.0s
[CV 5/5] END max_iter=50, penalty=l2, solver=lbfgs;, score=0.788 total time=   0.0s
[CV 5/5] END max_iter=50, penalty=l2, solver=newton-cg;, score=0.788 total time=   0.0s
[CV 1/5] END max_iter=50, penalty=l2, solver=newton-cg;, score=0.796 total time=   0.0s
[CV 1/5] END max_iter=50, penalty=l2, solver=liblinear;, score=0.796 total ti

/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://s

[CV 3/5] END max_iter=50, penalty=none, solver=lbfgs;, score=0.740 total time=   0.1s
[CV 4/5] END max_iter=50, penalty=none, solver=lbfgs;, score=0.722 total time=   0.1s
[CV 5/5] END max_iter=50, penalty=none, solver=lbfgs;, score=0.750 total time=   0.1s
[CV 1/5] END max_iter=50, penalty=none, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END max_iter=50, penalty=none, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END max_iter=50, penalty=none, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END max_iter=50, penalty=none, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END max_iter=50, penalty=none, solver=liblinear;, score=nan total time=   0.0s
[CV 1/5] END max_iter=50, penalty=none, solver=sag;, score=0.725 total time=   0.0s
[CV 2/5] END max_iter=50, penalty=none, solver=sag;, score=0.736 total time=   0.0s
[CV 3/5] END max_iter=50, penalty=none, solver=sag;, score=0.755 total time=   0.0s
[CV 4/5] END max_iter=50, penalty=none, solver=sag

/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarni

[CV 3/5] END max_iter=50, penalty=none, solver=saga;, score=0.757 total time=   0.0s
[CV 1/5] END max_iter=50, penalty=elasticnet, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END max_iter=50, penalty=elasticnet, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END max_iter=50, penalty=elasticnet, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END max_iter=50, penalty=elasticnet, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END max_iter=50, penalty=none, solver=saga;, score=0.737 total time=   0.0s
[CV 5/5] END max_iter=50, penalty=elasticnet, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END max_iter=50, penalty=elasticnet, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END max_iter=50, penalty=elasticnet, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END max_iter=50, penalty=elasticnet, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END max_iter=50, penalty=none, solver=saga;, score=0.755 total time=   0.0s
[CV 1/5] END 

...

/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 3/5] END max_iter=1000, penalty=none, solver=saga;, score=0.731 total time=   0.6s
[CV 4/5] END max_iter=1000, penalty=none, solver=saga;, score=0.709 total time=   0.6s


/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 5/5] END max_iter=1000, penalty=none, solver=saga;, score=0.720 total time=   0.5s
[CV 1/5] END max_iter=1000, penalty=none, solver=lbfgs;, score=0.701 total time=   0.5s
[CV 3/5] END max_iter=1000, penalty=none, solver=newton-cg;, score=0.721 total time=   4.8s
[CV 4/5] END max_iter=1000, penalty=none, solver=newton-cg;, score=0.672 total time=   1.5s


GridSearchCV(cv=5, estimator=LogisticRegression(random_state=42), n_jobs=-1,
             param_grid={'max_iter': [50, 100, 200, 300, 400, 500, 600, 1000],
                         'penalty': ['l2', 'l1', 'none', 'elasticnet'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='accuracy', verbose=5)

In [23]:
lg_best_tf = gs_tf.best_estimator_
y_pred_test_tf = lg_best_tf.predict(X_test_tf)
print_pretty_summary('LG tf-idf model', lg_best_tf, y_test, y_pred_test_tf)

LG tf-idf model
penalty: l2
solver: newton-cg
max_iter: 50
Test accuracy: 0.793828


0.793827971109652

### **_GS word embedding_**

In [24]:
gs_ed = GridSearchCV(lg_classifier, param_grid, scoring='accuracy',
                  cv=5, verbose=5, n_jobs=-1)

In [25]:
gs_ed.fit(X_train_pca2, y_train2)

Fitting 5 folds for each of 160 candidates, totalling 800 fits


/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only supp

[CV 2/5] END max_iter=50, penalty=l2, solver=lbfgs;, score=0.802 total time=   0.3s
[CV 3/5] END max_iter=50, penalty=l2, solver=lbfgs;, score=0.786 total time=   0.4s
[CV 1/5] END max_iter=50, penalty=l2, solver=lbfgs;, score=0.783 total time=   0.4s


/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge

[CV 4/5] END max_iter=50, penalty=l2, solver=lbfgs;, score=0.786 total time=   0.3s
[CV 5/5] END max_iter=50, penalty=l2, solver=lbfgs;, score=0.772 total time=   0.3s


...

/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


[CV 4/5] END max_iter=1000, penalty=none, solver=saga;, score=0.786 total time=   7.5s
[CV 3/5] END max_iter=1000, penalty=none, solver=saga;, score=0.782 total time=   8.1s


/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


[CV 5/5] END max_iter=1000, penalty=none, solver=saga;, score=0.781 total time=   6.5s


GridSearchCV(cv=5, estimator=LogisticRegression(random_state=42), n_jobs=-1,
             param_grid={'max_iter': [50, 100, 200, 300, 400, 500, 600, 1000],
                         'penalty': ['l2', 'l1', 'none', 'elasticnet'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='accuracy', verbose=5)

In [26]:
lg_best_ed = gs_ed.best_estimator_
y_pred_test_ed = lg_best_ed.predict(X_test_pca2)
print_pretty_summary('LG word embedding model', lg_best_ed, y_test2, y_pred_test_ed)

LG word embedding model
penalty: l1
solver: saga
max_iter: 200
Test accuracy: 0.808930


0.8089297439264609

## **KMean Clustering**

Seems doesn't work 

In [27]:
from sklearn.cluster import MiniBatchKMeans, KMeans, AgglomerativeClustering

In [28]:
kmeans = MiniBatchKMeans(n_clusters=2, verbose=0, init='k-means++', max_iter=150, n_init=20, random_state=RSEED)
kmeans.fit(X_pca)

MiniBatchKMeans(max_iter=150, n_clusters=2, n_init=20, random_state=42)

In [29]:
print('Rand Index: {}'.format(rand_score(df.target, kmeans.labels_)))

Rand Index: 0.5027081549185131
